In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
df_reviews = pd.read_feather("../FilteredData/review_philly.feather")
df_users = pd.read_feather("../FilteredData/user_philly.feather")
df_buz = pd.read_feather("../FilteredData/business_philly.feather")

In [3]:
df_reviews.columns

Index(['_id', 'review_id', 'user_id', 'business_id', 'stars', 'useful',
       'funny', 'cool', 'text', 'date', 'compliment_count'],
      dtype='object')

In [4]:
df_users.columns

Index(['index', '_id', 'user_id', 'name', 'review_count', 'yelping_since',
       'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars',
       'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note',
       'compliment_plain', 'compliment_cool', 'compliment_funny',
       'compliment_writer', 'compliment_photos', 'gender_score',
       'philly_reviews', 'philly_share_of_reviews'],
      dtype='object')

In [5]:
df_buz.columns

Index(['index', '_id', 'business_id', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours', 'positive_%'],
      dtype='object')

In [6]:
df_users.rename(columns={"review_count":"user_review_count"}, inplace=True)

In [7]:
df_buz.rename(columns={"stars":"business_stars", "review_count":"business_review_count"}, inplace=True)

__user_features:__
* user_review_count (numerical)
* average_stars (numerical)
* gender_score (numerical)
* philly_reviews (numerical)
* philly_share_of_reviews (numerical)
* years_on_yelp ?
    
    
__item_features:__
* postal_code (categorical)
* business_stars (numerical)
* business_review_count (numerical)
* <s>positive_% (numerical)</s>
* attributes?
* categories?

In [8]:
df_buz["positive_%"].isna().sum()

541

In [26]:
df_buz["postal_code"] = df_buz["postal_code"].fillna(0)

In [27]:
df_reviews[df_reviews.stars.isnull()]

,_id,review_id,user_id,business_id,stars,useful,funny,cool,text,date,compliment_count
852117,6327b024504265271679c187,nan,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,NaN,NaN,NaN,NaN,It's open even when you think it isn't,2013-08-18 00:56:08,0.0
852118,6327b024504265271679c199,nan,FQ-zmWPEG_pjSQx6pt3Efw,3ZynJ94VpIdDlaArmEp2Rg,NaN,NaN,NaN,NaN,"Yes, I'm eating here again. Breakfast!",2012-10-12 15:16:13,0.0
852119,6327b024504265271679c1a7,nan,YnlCpuaBa3qWBp4te8pGmA,XIKYdKWq72zUYsq8NBxcCQ,NaN,NaN,NaN,NaN,The honey glazed salmon is amazing!,2018-01-14 15:00:01,0.0
852120,6327b024504265271679c1b5,nan,Rr4cLb6Go91FT134o6RsKg,eMiN8nm70jjKg8izikVWDA,NaN,NaN,NaN,NaN,Mmm Yummy Crab Fries!,2011-10-16 23:43:10,0.0
852121,6327b024504265271679c1ba,nan,fJhr0G2JBNkfqpbIwkEQHg,eJ77e9lGxY3ArzaoDbHhYw,NaN,NaN,NaN,NaN,"Good specials, nice menu",2013-02-16 21:34:37,0.0
...,...,...,...,...,...,...,...,...,...,...,...
855075,6327b03050426527167a1746,nan,uChvFHK4uSifUXBPJPp9Zw,lCetcbxQr52TTGmGmXzQcw,NaN,NaN,NaN,NaN,Racist!!! Wont serve you if you are ethnic,2013-12-15 04:51:26,1.0
855076,6327b03050426527167a1749,nan,lP_cx94U8stOuQ8HrGecdQ,03jQGGJ2ch0uHTtW-UUUqg,NaN,NaN,NaN,NaN,Stuff face time.,2012-05-27 14:06:04,0.0
855077,6327b03050426527167a174c,nan,Rr4cLb6Go91FT134o6RsKg,UEGZ1nHUVb2ltGtd7C7X9w,NaN,NaN,NaN,NaN,Blizzard,2012-06-05 20:34:27,0.0
855078,6327b03050426527167a175b,nan,pflneuKFKQKQvZGDxebGFw,zsaDvomuMLe_-Ibtk9sA-A,NaN,NaN,NaN,NaN,Stopped by and it was closed on wed. 10/3/12!,2012-10-03 16:33:53,0.0


In [28]:
df_reviews = df_reviews[~df_reviews.stars.isna()]

In [29]:
df_users.replace([np.inf, -np.inf], 1, inplace=True)

In [30]:
features = ["user_id", "business_id", "stars", "target",
            "user_review_count", "average_stars", "gender_score", "philly_reviews", "philly_share_of_reviews",
           "postal_code", "business_stars", "business_review_count"]

In [31]:
df_merged = df_reviews.merge(
    df_buz, 
    how="left", 
    left_on="business_id", 
    right_on="business_id"
)
df_merged = df_merged.merge(
    df_users,
    how="left",
    left_on="user_id",
    right_on="user_id"
)

In [32]:
def create_target(row):
    if row["stars"] <= 3:
        return 0
    else:
        return 1

In [33]:
df_merged["target"] = df_merged.apply(lambda x: create_target(x), axis=1)

In [34]:
df_merged = df_merged[features]

In [35]:
df_merged.shape[0] == df_reviews.shape[0]

True

In [36]:
df_train, df_val = train_test_split(
    df_merged, 
    test_size=0.2, 
    shuffle=True, 
    random_state=3
)

In [37]:
df_train.to_csv("./data/train.csv", index=False)
df_val.to_csv("./data/valid.csv", index=False)

In [38]:
df_train = pd.read_csv("./data/train.csv")
df_train.shape

(681693, 12)